In [ ]:
# get voronoi boundaries

# get numpy array of image
#   - get centers of nuclei in numpy array
#   - prediction.boxes.xywh.numpy()
#       - export to csv
#           - how do you export to csv?


In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
from ultralytics import YOLO
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
from IPython.display import clear_output
import socket
import time
from datetime import datetime
import shutil
from random import randint
import re
import imagecodecs
import tifffile
import tensorflow as tf
from scipy.spatial import Voronoi, cKDTree
from matplotlib.path import Path
from rtree import index
from ctypes.wintypes import INT
import time
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import torch, torchvision

print("torchvision: ", torchvision.__version__)
print('CUDA available:', torch.cuda.is_available())

torchvision:  0.17.2
CUDA available: True


In [2]:
yolo_model_path = r"S:\Phys\FIV925 XSection\Datasets\Axons vs SCs\01i5\YO 384 0530 MAXI\map75=0855192 YOLOv8n-seg map75=081193 idx=4 ep=24 btch=32 rnd=2840511\weights\best.pt"

model = YOLO(yolo_model_path)

In [ ]:
tile_size = 384
iteration = 0
scale_factor = 1
dataset = "Human Lung Cancer"
base_path = r"R:\FIVE\EXP\FIV925\Additional Datasets"
ome_tiff_path = f"{base_path}\\{dataset}\\morphology_mip.ome.tif"

with tifffile.TiffFile(ome_tiff_path) as tif:
    image = tif.asarray()
    # image_dapi = image[2,:, :] # DAPI channel
    image_data = image[::2, ::2] # downsample by half

    # normalize to 8 bit
    max_val_16bit = np.max(image_data)
    image_scaled = (image_data / max_val_16bit) * 255
    image_8bit = image_scaled.astype(np.uint8)

    # create image and resize
    og_image = Image.fromarray(image_8bit, 'L')
    new_width = int(og_image.width * scale_factor); new_height = int(og_image.height * scale_factor)
    image_resized = og_image.resize((new_width, new_height), Image.LANCZOS)
    image = np.array(image_resized)

    image_height, image_width = image_resized.size

    big_array = np.zeros(image_resized.size)
    list_xywh = []

    # as we're making predictions, we want a big array that's meant to 'contain' the whole image
    #   then insert the predictions into that big array
    # so here is where we go through the image as tiles and make the predictions
    for i in range(0, image_height, 384): # section_miny, section_maxy
        for j in range(0, image_width, 384): # section_minx, section_maxx
            tile = image[i:i+tile_size, j:j+tile_size]
            
            if tile.shape[0] == 0 or tile.shape[1] == 0:
                continue  # Skip tiles that have no width or height
            tile_as_img = Image.fromarray(tile, 'L')
            prediction = model.predict(tile_as_img, show=False, max_det=3000)
            
            # trying to make the numpy array with the centers of the boxes
            #   theoretically, this should a numpy array of size 384x384 (in most cases)
            #   wrong, this is a list of the xywh coords
            # small_array = prediction[0].boxes.xywh.cpu().numpy()
            # big_array[i:i+small_array.shape[0], j:j+small_array.shape[1]] = small_array
            list_xywh.append(prediction[0].boxes.xywh.cpu().numpy())
            iteration += 1
            print(iteration)
    
            points = list_xywh
            
            # from Lina's code
            # if len(points) < 4:
            #     continue
            if len(set(points[:, 0])) > 1 and len(set(points[:, 1])) > 1:
                vor = Voronoi(points)
            else:
                continue
            region_paths = {}
            transcript_voronoi_regions = []

            for point_region_id, region_id in enumerate(vor.point_region):
                if (-1 not in vor.regions[region_id]):
                    region_paths[point_region_id] = Path(vor.vertices[vor.regions[region_id]])
            region_paths = {}
            voronoi_regions = []

            for point_region_id, region_id in enumerate(vor.point_region):
                if (-1 not in vor.regions[region_id]):
                    region_paths[point_region_id] = Path(vor.vertices[vor.regions[region_id]])

            ## For each transcript point, identify which Voronoi cell it falls into
            for point in points:
                point_region_id = -1  # Default to None if no region contains the point
                for region_id, region_path in region_paths.items():
                    if region_path.contains_point(point):
                        point_region_id = region_id
                        break  # Stop checking if we found the region
                voronoi_regions.append(point_region_id)
            # end of Lina's code    
            
            # big_array[i:i+tile.shape[0], j:j+tile.shape[1]] = voronoi_regions
                        
# Save the array with a header and formatted floating-point numbers
# np.savetxt(r"R:\FIVE\EXP\FIV925\Additional Datasets\Human Lung Cancer\array.csv", big_array, delimiter=",", fmt='%.2f')

    

c:\Users\FIVE\anaconda3\envs\torch01\lib\site-packages\torch\nn\modules\module.py:1150: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/cuda/CUDAAllocatorConfig.h:30.)
  return t.to(device, dtype if t.is_floating_point() or t.is_complex() else None, non_blocking)


0: 384x384 (no detections), 10.0ms
Speed: 3.0ms preprocess, 10.0ms inference, 16.4ms postprocess per image at shape (1, 3, 384, 384)


: 

In [4]:
ar = prediction[0].boxes.xywh.cpu().numpy()

In [5]:
print(ar)

[[ 35.660927  268.86853    13.50577    12.989868 ]
 [ 23.839218  276.08948    13.931564   13.951111 ]
 [100.46495   182.72694    10.622742   11.629486 ]
 ...
 [280.11145   130.82216    12.093018    9.311768 ]
 [  2.3115494 364.06012     4.5220504   9.036438 ]
 [237.07748    78.93929     9.006226   12.2500305]]
